# CVRP
Suponga que correos de chile debe decidir cuantos vehículos debe utilizar para distribuir paquetes en Antofagasta en un día cualquiera. Considere los siguientes aspectos:
- Se tiene disponible, al inicio de cada día, una lista de paquetes que necesariamente deben ser entregados.
- Cada paquete tiene un peso y volumen conocido.
- Se cuenta con una serie de vehículos con capacidad en peso y volumen conocidos

Considere los siguientes parametros:
* $C$ conjunto de clientes o paquetes
* $N=\{0\}\cup N$ conjunto de nodos, donde $0$ representa al depot
* $A=\{(i,j)\in N^2 : i\neq j\}$ conjunto de todos las conexiones (arcos) entre nodos
* $d_{i,j}$ distancia entre los nodos $i$ y $j$, con $(i,j)\in A$
* $p_i$ peso de cada paquete $i\in N$
* $v_i$ volumen de cada paquete $i\in N$
* $V$ capacidad de volumen de cada vehículo
* $P$ capacidad de peso de cada vehículo

Considere las siguientes variables de decisión:
* $x_{i,j}$ igual a uno cuando se decide viajar sobre el arco $(i,j)\in A$; igual a cero de lo contrario 
* $u_i$ volumen acumulado en una ruta hasta el cliente $i\in C$
* $w_j$ peso acumulado en una ruta hasta el cliente $i\in C$

Luego, la formulación matemática del problema es:
\begin{align}
\min \quad & \sum_{(i,j)\in A}d_{i,j}x_{i,j} \\
\text{s.t.} \quad & \sum_{(i,j)\in A : j=h} x_{i,j} = 1 && \forall \ h \in C \\
& \sum_{(i,j)\in A : i=h} x_{i,j} = 1 && \forall \ h \in C \\
& x_{i,j} = 1 \Rightarrow u_i + v_j = u_j && \forall \ (i,j) \in A : i\neq 0 , \ j\neq 0 \\
& x_{i,j} = 1 \Rightarrow w_i + p_j = w_j && \forall \ (i,j) \in A : i\neq 0 , \ j\neq 0 \\
& v_i \leq u_i \leq V && \forall \ i \in C \\
& p_i \leq w_i \leq P && \forall \ i \in C \\
& x_{i,j} \in \{0,1\} && \forall \ (i,j) \in A
\end{align}

In [1]:
import numpy as np
import xlwings as xw
import folium
import docplex.mp.solution as mp_sol
from docplex.mp.model import Model
from geopy.distance import great_circle

In [2]:
n = 15
paquetes = [i for i in range(1, n + 1)]
nodos = [0] + paquetes

In [3]:
arcos = [(i,j) for i in nodos for j in nodos if i != j]

In [4]:
lats = xw.Range('E2:E%d'%(n+2)).value
lons = xw.Range('F2:F%d'%(n+2)).value
distancia = {(i,j): great_circle((lats[i], lons[i]), (lats[j], lons[j])).meters for i, j in arcos}
volumen = xw.Range('G2:G%d'%(n+2)).value
peso = xw.Range('H2:H%d'%(n+2)).value
volumen_max = xw.Range('K1').value
peso_max = xw.Range('K2').value

In [5]:
mdl = Model('CVRP Correos de Chile')

In [6]:
x = mdl.binary_var_dict(arcos, name='x')
u = mdl.continuous_var_dict(paquetes, name='u')
w = mdl.continuous_var_dict(paquetes, name='w')

$$\min \sum_{(i,j)\in A}d_{i,j}x_{i,j}$$

In [7]:
mdl.minimize(mdl.sum(distancia[(i, j)] * x[(i, j)] for i, j in arcos))

$$ \sum_{(i,j)\in A : j=h} x_{i,j} = 1 \qquad \forall \ h \in C $$

In [8]:
for h in paquetes:
    mdl.add_constraint(mdl.sum(x[(i,j)] for i,j in arcos if i==h)==1, ctname='out_%d'%h)

$$ \sum_{(i,j)\in A : i=h} x_{i,j} = 1 \qquad \forall \ h \in C $$

In [9]:
for h in paquetes:
    mdl.add_constraint(mdl.sum(x[(i,j)] for i,j in arcos if j==h)==1, ctname='in_%d'%h)

$$ x_{i,j} = 1 \Rightarrow u_i + v_j = u_j \qquad \forall \ (i,j) \in A : i\neq 0 , \ j\neq 0 $$
$$ x_{i,j} = 1 \Rightarrow w_i + p_j = w_j \qquad \forall \ (i,j) \in A : i\neq 0 , \ j\neq 0 $$

In [10]:
for i, j in arcos:
    if i!=0 and j!=0:
        mdl.add_indicator(x[(i,j)], u[i] + volumen[j] == u[j])
        mdl.add_indicator(x[(i,j)], w[i] + peso[j] == w[j])

$$ v_i \leq u_i \leq V \qquad \forall \ i \in C $$
$$ p_i \leq w_i \leq P \qquad \forall \ i \in C $$

In [11]:
for i in paquetes:
    mdl.add_constraint(volumen[i] <= u[i])
    mdl.add_constraint(u[i] <= volumen_max)
    mdl.add_constraint(peso[i] <= w[i])
    mdl.add_constraint(w[i] <= peso_max)

In [12]:
mdl.parameters.timelimit = 60
mdl.parameters.mip.tolerances.mipgap = 0.1
mdl.parameters.mip.strategy.branch = 1
solucion = mdl.solve(log_output=True)

CPXPARAM_TimeLimit                               60
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Read_APIEncoding                        "UTF-8"
CPXPARAM_MIP_Strategy_Branch                     1
CPXPARAM_MIP_Tolerances_MIPGap                   0.10000000000000001
CPXPARAM_MIP_Strategy_CallbackReducedLP          0
Found incumbent of value 166726.698961 after 0.00 sec. (0.02 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 60 rows and 0 columns.
MIP Presolve modified 210 coefficients.
Aggregator did 210 substitutions.
Reduced MIP has 240 rows, 480 columns, and 1080 nonzeros.
Reduced MIP has 240 binaries, 0 generals, 0 SOSs, and 420 indicators.
Presolve time = 0.02 sec. (1.50 ticks)
Probing time = 0.00 sec. (1.83 ticks)
Tried aggregator 1 time.
Reduced MIP has 240 rows, 480 columns, and 1080 nonzeros.
Reduced MIP has 240 binaries, 0 generals, 0 SOSs, and 420 indicators.
Presolve time = 0.00 sec. (0.82 ticks)
Probing time = 0.00 sec. (1.77 ticks)
Clique table member

In [13]:
mdl.get_solve_status()

<JobSolveStatus.FEASIBLE_SOLUTION: 1>

In [14]:
# Encontrar las rutas por separado
rutas = []
for h in paquetes:
    if x[(0, h)].solution_value > 0.9:
        arcos_ruta = [(0, h)]
        j = h
        while j!=0:
            i = j
            for k in nodos:
                if i!=k and x[(i, k)].solution_value > 0.9:
                    j = k
                    arcos_ruta.append((i, j))
                    break
        rutas.append(arcos_ruta)
rutas

[[(0, 1), (1, 9), (9, 11), (11, 5), (5, 0)],
 [(0, 2), (2, 10), (10, 13), (13, 0)],
 [(0, 8), (8, 6), (6, 7), (7, 3), (3, 0)],
 [(0, 15), (15, 14), (14, 12), (12, 4), (4, 0)]]

In [15]:
mapa = folium.Map(location=xw.Range('E2:F2').value, zoom_start=12)

fg = folium.FeatureGroup(name='Direcciones')

# Graficar el depot
lat, lon = xw.Range('E2:F2').value
fg.add_child(folium.Marker(location=[lat,lon],
                           popup=folium.Popup('Depot'),
                           icon=folium.Icon(color='red', 
                                            icon_color='white')
                           ))

# Graficar los clientes
for nombre, dire, status, tipo, lat, lon in xw.Range('A3:F%d'%(n+2)).value:
    fg.add_child(folium.Marker(location=[lat,lon],
                               popup=folium.Popup(nombre),
                               icon=folium.Icon(color='blue',
                                                icon_color='white')
                               ))

mapa.add_child(fg)

# Graficar las rutas por separado
k = 0
for ruta in rutas:
    k = k + 1
    fg = folium.FeatureGroup(name='Ruta %d'%(k))
    for i, j in ruta:
        poly = folium.PolyLine(locations=[[lats[i],lons[i]],[lats[j],lons[j]]], weight=5)
        fg.add_child(poly)
    mapa.add_child(fg)

# Mostrar el mapa
mapa.add_child(folium.LayerControl())
mapa